## Quantifying Linguistic Degeneration

Once I have the data, I must think about how to quantify linguistic degeneration. For this, I will be using the following metrics:
<br>

***Lexical Quality***

1. Type-token ratio (TTR): measures vocabulary diversity
2. Zipf curve tail heaviness: measures use of rare words
3. Average word length: measures use of larger words
4. Age of acquisition: measures word difficulty

***Syntactic Quality***

5. Parse tree depth: measures sentence complexity
6. Fragment ratio: measure of informality
7. Index of Syntactic Complexity: measures overall syntactic quality

***Orthographical Quality***

8. Levenshtein distance to dictionary: spelling errors
9. Punctuation frequency: attention to grammar

***Substantive Quality***

10. Abstract concepts
11. Figurative language

Note that I do not use other traditional measures of lexical complexity, such as ambiguity, vague quantifier frequency, orthographic neighborhood size, terminology inconsistency. Word ambiguity almost exclusively measures the complexity of the text from the perspective of the reader. As the purpose of this analysis is to look at language degeneration over time as a result of modern phenomena, I am principally interested in the intellect required to produce the complexity, meaning I am looking at true substantive complexity as opposed to inteprative complexity. Similarly, vague quantifier frequency, orthographic neighborhood size, and terminology inconsistency also measure complexity of interpratation, hence they are excluded. It is also worth noting that I categorize abstract concepts and figurative language under substantive quality as opposed to lexical complexity, as they involve more intimately the meaning of the word.

Likewise, I exclude certain measures of syntactic complexity, such as sentence length, information overload, passive voice, and negation. In the case of sentence length, it can be a sign of high (i.e., complexity) or low (i.e., run-on sentences) quality writing, which is why I have excluded it. Additionally, sentence complexity can be better measured via ICS. Information overload, passive voice, and negation are excluded for the same interpratability versus quality distinction in the above paragraph.